# Homework 4: Language Modelling in Hangman

## General info

<b>Due date</b>: 11pm, Wednesday May 2nd

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models, ranging from unigram approaches to higher over n-gram models. Your objective is to create an automatic player which makes the fewest mistakes.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [2]:
# hangman('tea', human, 8, True)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(0.5 mark)

In [3]:
import nltk
from nltk.corpus import brown
import re
import numpy as np

In [4]:
word_set = set()
for word in brown.words():
    if re.match(r"^[a-zA-Z]+$", word):
        word = word.lower()
        if word not in word_set:
            word_set.add(word)

In [5]:
word_list = np.asarray(list(word_set))
np.random.shuffle(word_list)

test_set = word_list[:1000]
train_set = word_list[1000:]

print("Length of test set is:", len(test_set))
print("Length of train set is:", len(train_set))

Length of test set is: 1000
Length of train set is: 39234


In [6]:
# hangman(np.random.choice(test_set), human, 8, True)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To measure the performance of this (and later) techiques, implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. You will want to turn off the printouts for this, using the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

(1 mark)

In [7]:
def random_guesser(mask, guessed, **kwargs):
    alphabet = {chr(i) for i in range(ord('a'), ord('z')+1)}
    return np.random.choice(list(alphabet - guessed))

In [8]:
hangman(np.random.choice(test_set), random_guesser, 26, True)

Starting hangman game. Target is _ _ _ _ _ _ _ length 7
You have 26 attempts remaining.
Guess is s
Good guess: _ _ _ _ _ _ s
You have 26 attempts remaining.
Guess is d
Sorry, try again.
You have 25 attempts remaining.
Guess is b
Sorry, try again.
You have 24 attempts remaining.
Guess is h
Sorry, try again.
You have 23 attempts remaining.
Guess is e
Good guess: _ _ _ _ e _ s
You have 23 attempts remaining.
Guess is j
Sorry, try again.
You have 22 attempts remaining.
Guess is a
Good guess: _ a _ _ e _ s
You have 22 attempts remaining.
Guess is c
Sorry, try again.
You have 21 attempts remaining.
Guess is l
Good guess: _ a _ _ e l s
You have 21 attempts remaining.
Guess is u
Sorry, try again.
You have 20 attempts remaining.
Guess is r
Good guess: _ a r _ e l s
You have 20 attempts remaining.
Guess is p
Sorry, try again.
You have 19 attempts remaining.
Guess is g
Sorry, try again.
You have 18 attempts remaining.
Guess is y
Sorry, try again.
You have 17 attempts remaining.
Guess is i
Sorry, 

13

In [9]:
def measure_performance(guesser, max_mistakes=26, **guesser_args):
    mistake_num = 0
    count = 0.0
    
    for test_word in test_set:
        mistake_num += \
            hangman(test_word, guesser, max_mistakes, False, **guesser_args)
        count += 1
    
    return mistake_num / count

In [10]:
print("Average number of mistakes for the random AI is:", 
     measure_performance(random_guesser))

Average number of mistakes for the random AI is: 16.746


**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. Remember to exclude already guessed characters, and use the same evaluation method as above, so the results are comparable. (Hint: it should be much lower than for random).

(1 mark)

In [11]:
from collections import Counter


def get_counts(words):
    unigram_counts = Counter()

    # collect unigram statistics
    for word in words:
        for char in word:
            unigram_counts[char] += 1
        
    return unigram_counts


def unigram_guesser(mask, guessed, **kwargs):
    for (char, _) in kwargs["unigrams"].most_common():
        if char not in guessed:
            return char

In [12]:
unigrams = get_counts(train_set)
print(unigrams)

Counter({'e': 35251, 'i': 26012, 'a': 24397, 's': 23729, 'n': 22591, 'r': 22074, 't': 20672, 'o': 19219, 'l': 16714, 'c': 12623, 'd': 12281, 'u': 10162, 'm': 8686, 'g': 8605, 'p': 8534, 'h': 7337, 'b': 5749, 'y': 5355, 'f': 4228, 'v': 3442, 'k': 2972, 'w': 2797, 'z': 1020, 'x': 832, 'j': 641, 'q': 527})


In [13]:
print("Average number of mistakes for the unigram AI is:", 
      measure_performance(unigram_guesser, unigrams=unigrams))

Average number of mistakes for the unigram AI is: 10.522


**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Your job now is to incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. Create another AI guessing function using this new model, and print its test performance.   

(0.5 marks)

In [14]:
from collections import defaultdict

def get_conditional_counts(words):
    unigram_counts = defaultdict(Counter)
    # collect unigram statistics
    for word in words:
        for char in word:
            unigram_counts[len(word)][char] += 1
        
    return unigram_counts

In [15]:
conditional_unigrams = get_conditional_counts(train_set)

In [16]:
def conditional_unigram_guesser(mask, guessed, **kwargs):
    alphabet = {chr(i) for i in range(ord('a'), ord('z')+1)}
    if not len(mask) in kwargs["unigrams"]:
        return np.random.choice(list(alphabet - guessed))
    
    for (char, _) in kwargs["unigrams"][len(mask)].most_common():
        if char not in guessed:
            return char

    return np.random.choice(list(alphabet - guessed))

In [17]:
print("Average number of mistakes for the unigram AI is:", 
      measure_performance(conditional_unigram_guesser, unigrams=conditional_unigrams))

Average number of mistakes for the unigram AI is: 10.419


**Instructions:** Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

Your job is to develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) You should use linear interpolation to smooth between the higher order and lower order models, and you will have to decide how to weight each component. 

Your guessing AI algorithm should apply your language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

Implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set. Do you see any improvement over the unigram methods above?

(2 marks)

In [18]:
def get_ngram_counts(words):
    
    def convert_word(word, n):
        return "*" * (n - 1) + word
    
    counter_dict = dict()
    
    for n in range(2, 7):
        counter_dict[n] = defaultdict(Counter)
        for word in words:
            word = convert_word(word, n)
            ngram_list = [word[index:index+n] for index, _ in enumerate(word[:-n + 1])]
            for ngram in ngram_list:
                first, second = ngram[:-1], ngram[-1]
                counter_dict[n][first][second] += 1
    
    return counter_dict

In [19]:
ngrams = get_ngram_counts(train_set)
ngrams[1] = unigrams

In [20]:
import math


def get_interp(prev_chars, char, ngram_counts, lambdas):
    n = len(prev_chars) + 1
    
    if n == 1:
        return ngram_counts[n][char] / float(sum(ngram_counts[n].values()))

    total_count = float(sum(ngram_counts[n][prev_chars].values()))
    interp_prob = 0.0
    
    if total_count != 0:
        interp_prob = ngram_counts[n][prev_chars][char] / total_count
    
    return lambdas * interp_prob + \
        (1 - lambdas) * get_interp(prev_chars[1:], char, ngram_counts, lambdas)

    
def get_char_log_prob_interp(chars_list, char, n, ngram_counts, lambdas):
    log_prob = 0
    
    for index, chars in enumerate(chars_list):
        if len(chars) >= n:
            chars = chars[-n + 1:]
        log_prob += math.log(get_interp(chars, char, ngram_counts, lambdas) + 1)
    
    return log_prob
      
    
def trigram_guesser(mask, guessed, **kwargs):
    mask = "**" + "".join(mask)
    chars_list = mask.split("_")
    chars_list = chars_list[:-1]
    char_prob = dict()
    
    for i in [chr(c) for c in range(ord('a'), ord('z')+1)]:
        if not i in guessed:
            char_prob[i] = \
                get_char_log_prob_interp(chars_list, i, 3, kwargs["ngrams"], 0.9)
    return max(char_prob, key=char_prob.get)

In [21]:
print("Average number of mistakes for the trigram AI is:", 
      measure_performance(trigram_guesser, ngrams=ngrams))

Average number of mistakes for the trigram AI is: 8.234


In [22]:
def fourgram_guesser(mask, guessed, **kwargs):
    mask = "***" + "".join(mask)
    chars_list = mask.split("_")
    chars_list = chars_list[:-1]
    char_prob = dict()
    
    for i in [chr(c) for c in range(ord('a'), ord('z')+1)]:
        if not i in guessed:
            char_prob[i] = \
                get_char_log_prob_interp(chars_list, i, 4, kwargs["ngrams"], 0.9)
    return max(char_prob, key=char_prob.get)

print("Average number of mistakes for the fourgram AI is:", 
      measure_performance(fourgram_guesser, ngrams=ngrams))

Average number of mistakes for the fourgram AI is: 7.668


In [23]:
def fivegram_guesser(mask, guessed, **kwargs):
    mask = "****" + "".join(mask)
    chars_list = mask.split("_")
    chars_list = chars_list[:-1]
    char_prob = dict()
    
    for i in [chr(c) for c in range(ord('a'), ord('z')+1)]:
        if not i in guessed:
            char_prob[i] = \
                get_char_log_prob_interp(chars_list, i, 5, kwargs["ngrams"], 0.9)
    return max(char_prob, key=char_prob.get)

print("Average number of mistakes for the fivegram AI is:", 
      measure_performance(fivegram_guesser, ngrams=ngrams))

Average number of mistakes for the fivegram AI is: 7.484


## Bonus: Improving the AI

**Instructions:** To get the extra credit, you should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word and the parts that are known, fancier smoothing methods, using backwards ngram models, or a fancier inference algorithm like forward-back using a HMM. Ensure you report the test performance of your method.

You will be marked based on the ambition of your approach and on its accuracy. If you have tried some truly spectacular method but it didn't really work, then please include your implementation and an explanation, which will still attract marks for ambition.

(1 bonus mark) 

First, let's try sixgram:

In [24]:
def sixgram_guesser(mask, guessed, **kwargs):
    mask = "****" + "".join(mask)
    chars_list = mask.split("_")
    chars_list = chars_list[:-1]
    char_prob = dict()
    
    for i in [chr(c) for c in range(ord('a'), ord('z')+1)]:
        if not i in guessed:
            char_prob[i] = \
                get_char_log_prob_interp(chars_list, i, 6, kwargs["ngrams"], 0.9)
    return max(char_prob, key=char_prob.get)

print("Average number of mistakes for the sixgram AI is:", 
      measure_performance(sixgram_guesser, ngrams=ngrams))

Average number of mistakes for the sixgram AI is: 7.456


Try Backoff to smooth:

In [25]:
def get_backoff(prev_chars, char, ngram_counts, lambdas):
    n = len(prev_chars) + 1
    
    if n == 1:
        return ngram_counts[n][char] / float(sum(ngram_counts[n].values()))

    total_count = float(sum(ngram_counts[n][prev_chars].values()))
    
    if total_count != 0:
        return lambdas * ngram_counts[n][prev_chars][char] / total_count
    else:
        return (1 - lambdas) * get_interp(prev_chars[1:], char, ngram_counts, lambdas)

    
def get_char_log_prob_backoff(chars_list, char, n, ngram_counts, lambdas):
    log_prob = 0
    
    for index, chars in enumerate(chars_list):
        if len(chars) >= n:
            chars = chars[-n + 1:]
        log_prob += math.log(get_backoff(chars, char, ngram_counts, lambdas) + 1)
    
    return log_prob


def fivegram_backoff_guesser(mask, guessed, **kwargs):
    mask = "****" + "".join(mask)
    chars_list = mask.split("_")
    chars_list = chars_list[:-1]
    char_prob = dict()
    
    for i in [chr(c) for c in range(ord('a'), ord('z')+1)]:
        if not i in guessed:
            char_prob[i] = \
                get_char_log_prob_backoff(chars_list, i, 5, kwargs["ngrams"], 0.9)
    return max(char_prob, key=char_prob.get)

In [26]:
print("Average number of mistakes for the fivegram (backoff) AI is:", 
      measure_performance(fivegram_backoff_guesser, ngrams=ngrams))

Average number of mistakes for the fivegram (backoff) AI is: 7.842
